# Raster-based market access

Following in the footsteps of the Malaria Atlas Project's <a href="https://developers.google.com/earth-engine/datasets/catalog/Oxford_MAP_friction_surface_2015_v1_0">Global Friction Surface</a>, this notebook explores a set of tools for calculating:

1. Travel time rasters
2. Drive-time polygons
3. Marketsheds


In [1]:
import sys, os, importlib
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import GOSTnets as gn
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features
from shapely.geometry import box, Point, Polygon
from scipy.ndimage import generic_filter
from pandana.loaders import osm

sys.path.append("../")

import infrasap.market_access as ma
importlib.reload(ma)

<module 'infrasap.market_access' from '../infrasap/market_access.py'>

In [2]:
tutorial_folder = os.path.join("../", "tutorial_data")

dests = os.path.join(tutorial_folder, "destinations.shp")
friction_surface = os.path.join(tutorial_folder, "global_friction_surface.tif")

inD = gpd.read_file(dests)
inR = rasterio.open(friction_surface)
frictionD = inR.read()[0,:,:]
# convert friction surface to traversal time (lazily). Original data are
#    the original data are minutes to travel 1 m, so we will convert to 
#    minutes to cross the cell
frictionD = frictionD * 1000
mcp = graph.MCP_Geometric(frictionD)

In [3]:
ma.calculate_travel_time?

In [6]:
travel_costs, traceback = ma.calculate_travel_time(inR, mcp, inD)
travel_costs = travel_costs.astype(inR.meta['dtype'])
with rasterio.open(os.path.join(tutorial_folder, "least_cost_travel_time.tif"), 'w', **inR.meta) as out_file:
    out_file.write_band(1, travel_costs)

In [7]:
ma.generate_feature_vectors?

In [8]:
drive_time_thresholds = [60, 120, 180, 240] # minutes
drive_vectors = ma.generate_feature_vectors(inR, mcp, inD, drive_time_thresholds)
drive_vectors.to_file(os.path.join(tutorial_folder, "drive_vectors.shp"))

14:09:06	1 of 5
14:09:06	2 of 5
14:09:07	3 of 5
14:09:07	4 of 5
14:09:08	5 of 5


In [9]:
ma.generate_market_sheds?

In [10]:
importlib.reload(ma)
outfile = os.path.join(tutorial_folder, "marketsheds.tif")
ma.generate_market_sheds(inR, inD, out_file=outfile)

# Debugging belwo

In [31]:
ids = np.arange(traceback.size).reshape(traceback.shape)
def matrix_lambda(x):
    destination_coord = mcp.traceback([it.multi_index[0],it.multi_index[1]])[0]
    print(destination_coord)
    return(ids[destination_coord])

In [32]:
it = np.nditer(traceback, flags=['multi_index'])
result = []
for x in it:
    #print("%d <%s>" % (x, it.multi_index), end=' ')
    #matrix_lambda(x)
    result.append(matrix_lambda(x))

(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)


(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)


(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)


(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)


(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)


...

(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)


(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)


(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)


(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)
(237, 90)


KeyboardInterrupt: 

In [17]:
xx = np.array(result).reshape(traceback.shape)

In [22]:
outfile = os.path.join(tutorial_folder, "marketsheds_TOM.tif")
xx = xx.astype(np.byte)
meta = inR.meta.copy()
meta.update(dtype=xx.dtype)
with rasterio.open(outfile, 'w', **meta) as output:
    output.write_band(1, xx)

In [27]:
list(it)

[]

In [23]:
xx

array([[79, 79, 79, ..., 79, 79, 79],
       [79, 79, 79, ..., 79, 79, 79],
       [79, 79, 79, ..., 79, 79, 79],
       ...,
       [79, 79, 79, ..., 79, 79, 79],
       [79, 79, 79, ..., 79, 79, 79],
       [79, 79, 79, ..., 79, 79, 79]], dtype=int8)